![image](https://www.ewatercycle.org/assets/logo.png)

# Case study 1: replace subbasin in PCRGlobWB2.0 with a MARRMoT model
This notebooks demonstrates how to use eWaterCycle to combine the output of two very different models in a single experiment. We run PCRGlobWB2.0 for the Rhine basin, but we replace the Moselle sub-basin with a MARRMoT model.

PCRGlobWB2.0 (Edwin 20XX) is a distributed model written in python and MARRMoT (Knoben 20XX) is a suite of conceptual models written in Matlab. To make the difference as large as possible, we choose the simplest model available within MARRMoT: the m01, a single leaky bucket.

## Import statements
We'll be using the following modules

In [ ]:
# TODO: clean up code to consistently use the same convention! This is now a mess of 
# double imports because this notebook is constructed from two example notebooks 
# written by different team members at different times in the project.

#OS related 
from os import environ, remove
from os.path import abspath
import os
from pathlib import Path
from glob import glob
from configparser import ConfigParser
import pickle

#Time and time object related
import time
from datetime import datetime
from cftime import num2date

#calculations and plotting
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as sio
import xarray as xr
from scipy import optimize
from tqdm import tqdm

#cartography (drawing maps)
from cartopy.io import shapereader
from cartopy import crs

#hydrological specific non eWaterCycle
import hydrostats.metrics as hm
import hydrostats.visual as hv

#eWaterCycle specific
from ewatercycle.observation.grdc import get_grdc_data
from grpc4bmi.bmi_client_docker import BmiClientDocker

In [ ]:
from ewatercycle import CFG
from ewatercycle.models import MarrmotM01, PCRGlobWB
from ewatercycle.util import lat_lon_to_closest_variable_indices

In [ ]:
CFG.load_from_file('ewatercycle.yaml')

## List of files created for this experiment

### for PCRGlobWB2.0

- `rhine_05min_era5.ini` This file is identical to the file with the same name used in the big comparison study. The only difference is that the time period (variables startTime and endTime) have been set to 2002-01-01 and 2002-12-31 respectivly. 
- `rhine_05min_era5_without_moselle.ini` This file is identical to rhine_05in_era5.ini with the only exception that the landmask variable points to no_moselle_landmask_05min_rhine.map in the input directory
- `no_moselle_landmask_05min_rhine.map` This file is based on rhine_05min.map. Using a shape file with the shape of the Moselle catchment (see MARRMoT below) all pixels that are part of the moselle catchment have been removed (set to zero) in this landmask.

### for MARRMoT

- `marrmot_ERA5_Moselle_2001_2016.mat` This file contains the forcing that MARRMoT needs to run. It was created by running the ESMVAlTool recipe (TODO Jerom to provide details)

## Settings and parameters to run this experiment
The settings below are seperated in settings that 'belong' to the experiment, those that belong to PCRGlobWB or those that belong with MARRMoT

In [4]:
#Settings for this experiment

# The project home directory where data lives and output should be written
PROJECT_HOME = Path(os.path.abspath("/mnt/home/user36/temp/Case1"))
DATA_HOME = Path(os.path.abspath("/mnt/data/examples/technical_paper"))

# Settings for GRDC station for final comparison of streamflow
dstart = '2002-01-01'
dend = '2002-12-31'
station_id = '6335020' # GRDC station ID
basin_name = 'Rhine'

# Location of the mouth of the Moselle. The closest PCRGlobWB pixel to this location will
# be used to 'dump' the output of MARRMoT into the 'channel_storage' of PCRGlobWB. Location
# from Google maps.
latMoselleMouth = [50.366852]
lonMoselleMouth = [7.609666] 

In [5]:
#Settings for PCRGlobWB2.0
PCRGlob_input_dir=str(DATA_HOME / 'pcr-globwb' / 'input')

#TODO: timestamp these output directories to avoid problems
PCRGlob_Ref_output_dir=str(PROJECT_HOME / 'PCRGlobREFOutput')
PCRGlob_Exp_output_dir=str(PROJECT_HOME / 'PCRGlobExpOutput')

PCRGlob_Ref_setting_file = str(PROJECT_HOME  / 'settingFiles' / 'rhine_05min_era5.ini')
PCRGlob_Exp_setting_file = str(PROJECT_HOME / 'settingFiles' / 'rhine_05min_era5_without_moselle.ini')


In [6]:
#Settings for MARRMoT

# forcing dataset and directory
MARRMoT_FORCING_DIR = DATA_HOME / "marrmot" / "marrmotForcing" / "validation"
MARRMoT_forcing_file = "marrmot_ERA5_Moselle_2001_2016.mat"

#these two folders will be known inside the model-container as /data/input and /data/output
MARRMoT_container_in = PROJECT_HOME / 'inputMarrmot'
MARRMoT_container_out = PROJECT_HOME / 'outputMarrmot'

# this file will be created in 'containerIn' and contains the config setting for the model run. 
MARRMoT_config_file_name = 'config.mat'

# shape file that was used to delineate the catchment, is needed here to calculate area
shapefile = DATA_HOME / "marrmot"/ "marrmotForcing" / f"Moselle.shp"

#TODO: start en end time is repeated too often
# test period for both validation and calibration (Year, Month, Date)
period = {
    "spinup": datetime(2001, 1, 1,0,0,0),
    "start": datetime(2002, 1, 1,0,0,0),
    "end": datetime(2002, 12, 31,0,0,0),
}

# variable of interest to get out of the model
MARRMoT_output_variable = "flux_out_Q"

# flux_out_Q unit conversion factor from mm/day to m3/s
conversion_mmday2m3s = 1 / (1000 * 86400)

# parameters, in this case max soil moisture storage (in mm), ranging between 100 en 2000
# https://github.com/wknoben/MARRMoT/blob/dev-docker-BMI/MARRMoT/Models/Parameter%20range%20files
maximum_soil_moisture_storage = 1500.0
initial_soil_moisture_storage = 0.9 * maximum_soil_moisture_storage

## Derived directories and constants


In [7]:
# project directories for model input i.e. shape files and forcing
shape = shapereader.Reader(shapefile)
attr = next(shape.records())
moselle_area = attr.attributes["SUB_AREA"] * 1e6
print("The catchment area is:", moselle_area)

The catchment area is: 28101200000.0


## Setting up the model objects
For MARRMoT we first need to generate a config file. After that is done, the three BMI model objects are created

In [12]:
MARRMoT_model = MarrmotM01()

# Create config file and write to work directory (cfg_dir)
# Start up a container for MARRMoT
MARRMoT_cfg_file, MARRMoT_cfg_dir = MARRMoT_model.setup(
    forcing=MARRMoT_FORCING_DIR / MARRMoT_forcing_file,
    # No need to specifiy start and end date, using dates from forcing_output
    maximum_soil_moisture_storage=maximum_soil_moisture_storage,
    initial_soil_moisture_storage=initial_soil_moisture_storage,
)
MARRMoT_cfg_file, MARRMoT_cfg_dir

(PosixPath('/mnt/home/user36/temp/Case1/marrmot_20210423_143734/marrmot-m01_config.mat'),
 PosixPath('/mnt/home/user36/temp/Case1/marrmot_20210423_143734'))

In [13]:
# Initialize using the created config file
MARRMoT_model.initialize(str(MARRMoT_cfg_file))

In [14]:
# MARRMot needs to spin-up, so we run it for a while, see the period for timing
MARRMoT_time_units = MARRMoT_model.time_units
while num2date(MARRMoT_model.time, MARRMoT_time_units) < period["start"]:
    MARRMoT_model.update()

In [15]:
PCRGlob_Ref_model = PCRGlobWB()
# Start up a container for PCRGlob for the reference run 
PCRGlob_Ref_cfg_file, PCRGlob_Ref_cfg_dir = PCRGlob_Ref_model.setup(
    input_dir=PCRGlob_input_dir,
    cfg_file=PCRGlob_Ref_setting_file)
PCRGlob_Ref_cfg_file, PCRGlob_Ref_cfg_dir

Created working directory: /mnt/home/user36/temp/Case1/pcrglobwb_20210423_143813
Created config file /mnt/home/user36/temp/Case1/pcrglobwb_20210423_143813/pcrglobwb_ewatercycle.ini with inputDir /mnt/data/examples/technical_paper/pcr-globwb/input and outputDir /mnt/home/user36/temp/Case1/pcrglobwb_20210423_143813.
Started model container with working directory /mnt/home/user36/temp/Case1/pcrglobwb_20210423_143813 and access to the following input directories:
/mnt/data/examples/technical_paper/pcr-globwb/input.


(PosixPath('/mnt/home/user36/temp/Case1/pcrglobwb_20210423_143813/pcrglobwb_ewatercycle.ini'),
 PosixPath('/mnt/home/user36/temp/Case1/pcrglobwb_20210423_143813'))

In [16]:
# Initialize using the setting file
PCRGlob_Ref_model.initialize(str(PCRGlob_Ref_cfg_file))

In [17]:
# del PCRGlob_Ref_model.bmi
# del MARRMoT_model.bmi

In [18]:
sorted(PCRGlob_Ref_model.output_var_names)

['accumulated_land_surface_baseflow',
 'bare_soil_evaporation',
 'baseflow',
 'channel_storage',
 'consumptive_water_use_for_non_irrigation_demand',
 'desalination_source_abstraction',
 'desalination_source_abstraction_volume',
 'direct_runoff',
 'discharge',
 'domesticWaterConsumptionVolume',
 'domesticWaterWithdrawalVolume',
 'domestic_water_withdrawal',
 'evaporation_from_irrigation',
 'evaporation_from_irrigation_volume',
 'fossil_groundwater_abstraction',
 'fossil_groundwater_storage',
 'fraction_of_desalinated_water_allocation',
 'fraction_of_non_fossil_groundwater_allocation',
 'fraction_of_other_water_source_allocation',
 'fraction_of_surface_water',
 'fraction_of_surface_water_allocation',
 'groundwater_capillary_rise',
 'groundwater_recharge',
 'groundwater_storage',
 'industryWaterConsumptionVolume',
 'industryWaterWithdrawalVolume',
 'industry_water_withdrawal',
 'infiltration',
 'interception_evaporation',
 'interception_storage',
 'interflow',
 'irrigationWaterWithdrawalV

In [8]:
# Start up a container for PCRGlob for the experiment run 
PCRGlob_Exp_model = PCRGlobWB()
# Start up a container for PCRGlob for the reference run 
PCRGlob_Exp_cfg_file, PCRGlob_Exp_cfg_dir = PCRGlob_Exp_model.setup(
    input_dir=PCRGlob_input_dir,
    cfg_file=PCRGlob_Exp_setting_file)
PCRGlob_Exp_cfg_file, PCRGlob_Exp_cfg_dir

Created working directory: /mnt/home/user36/temp/Case1/pcrglobwb_20210423_143431
Created config file /mnt/home/user36/temp/Case1/pcrglobwb_20210423_143431/pcrglobwb_ewatercycle.ini with inputDir /mnt/data/examples/technical_paper/pcr-globwb/input and outputDir /mnt/home/user36/temp/Case1/pcrglobwb_20210423_143431.
Started model container with working directory /mnt/home/user36/temp/Case1/pcrglobwb_20210423_143431 and access to the following input directories:
/mnt/data/examples/technical_paper/pcr-globwb/input.


(PosixPath('/mnt/home/user36/temp/Case1/pcrglobwb_20210423_143431/pcrglobwb_ewatercycle.ini'),
 PosixPath('/mnt/home/user36/temp/Case1/pcrglobwb_20210423_143431'))

In [9]:
# Initialize using the setting file
PCRGlob_Exp_model.initialize(str(PCRGlob_Exp_cfg_file))

In [10]:
# Now that we have a BMI model for the experiment, we can query which model pixel is closest
# to the mouth of the Moselle, expressed in 'indices', ie. model pixel number.
Moselle_mouth_indices = lat_lon_to_closest_variable_indices(
    PCRGlob_Exp_model,
    'channel_storage',
    latMoselleMouth,
    lonMoselleMouth
)
Moselle_mouth_indices

array(6004)

In [19]:
#finally, create variables needed during the experiment run
PCRGlob_Ref_output = []
PCRGlob_Exp_output = []
MARRMot_output = []

PCRGlob_time_units = PCRGlob_Exp_model.time_units

## Running the experiment
With all pieces in place, we can now start to run the actual experiment. Each timestep first the reference model (PCRGlobWB 2.0 with the Moselle) is run and it's output stored. Subsequently, MARRMoT is run, it's output is stored and it is added to the experiment model (PCRGlobWB 2.0 without the Moselle). Than, finally, the experiment model is run for one timestep, it's output stored and we go on to the next timestep.

In [ ]:
print(f'Running experiment', flush=True)

#the experiment model is used as central 'time keeper'
while PCRGlob_Exp_model.time < PCRGlob_Exp_model.end_time:
    
    #get the current time to print
    time = num2date(PCRGlob_Exp_model.time, PCRGlob_time_units)
    print(f'Current time: {time}', end="\r")

    #run the reference model for one timestep, store the output
    PCRGlob_Ref_model.update()
    PCRGlob_Ref_output.append(PCRGlob_Ref_model.get_value_as_xarray("discharge"))
    
    #run MARRMoT and store the output, note that MARRMoT output is in mm!
    MARRMoT_model.update()
    MARRMot_output.append(MARRMoT_model.get_value(MARRMoT_output_variable)[0])
    
    #add the output of MARRMoT to PCRGlob Experiment model. Note that channel storage
    # is in m3, while MARRMoT output is in mm, so we need to convert
    water_to_add_to_PCRGlob = MARRMot_output[-1] * moselle_area / 1000 #now in m3
    value_to_set_in_PCRGlob = water_to_add_to_PCRGlob + PCRGlob_Exp_model.bmi.get_value_at_indices('channel_storage', Moselle_mouth_indices)
    PCRGlob_Exp_model.bmi.set_value_at_indices('channel_storage', Moselle_mouth_indices, value_to_set_in_PCRGlob)
    
    #Run the experiment PCRGlobWB model one timestep, store the output
    PCRGlob_Exp_model.update()
    PCRGlob_Exp_output.append(PCRGlob_Exp_model.get_value_as_xarray("discharge"))
    
print('')

Running experiment


### clean up after the model run
The models have to be 'finalized', which deletes any temporary files and the containers have to be shut down.

In [ ]:
PCRGlob_Ref_model.finalize()
PCRGlob_Exp_model.finalize()
MARRMoT_model.finalize()

del PCRGlob_Exp_model
del PCRGlob_Ref_model
del MARRMoT_model

In [ ]:
f = open('.\..\modelrunFull.pckl', 'wb')
pickle.dump([PCRGlob_Ref_output, PCRGlob_Exp_output, MARRMot_output],f)
f.close()
del f

## Import GRDC observations

Read the GRDC data for the period of the experiment, for the station given above. 
If you are not on jupyter.ewatercycle.org or cartesius, download the GRDC https://www.bafg.de/GRDC/EN/02_srvcs/21_tmsrs/riverdischarge_node.html and set the directory below in GRDC_DATA_HOME env var.

### TODO the four lines at the end of the cell should become part of the get_grdc_data function, which (I think) should return a dict.

In [ ]:
#on Cartesius:
#environ['GRDC_DATA_HOME']= '/projects/0/wtrcycle/GRDC/GRDC_GCOSGTN-H_27_03_2019'

#on jupyter.ewatercycle.org
environ['GRDC_DATA_HOME']= '/mnt/data/grdc/dailies' 

#TODO: 4 lines below need to be one line.

observations = get_grdc_data(station_id, start_date=dstart, end_date=dend)
grdc_obs = observations.to_dataframe().rename(columns = {'streamflow': 'GRDC'})
grdc_lon = observations.attrs['grdc_longitude_in_arc_degree']
grdc_lat = observations.attrs['grdc_latitude_in_arc_degree']

## Plot the results

In [ ]:
#f = open('.\..\modelrunFull.pckl', 'rb')
#data = pickle.load(f)
#f.close()
#del f

In [ ]:
dataExp  = xr.concat(PCRGlob_Exp_output, dim='time')
dataRef  = xr.concat(PCRGlob_Ref_output, dim='time')

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 10), dpi=120)

dataExp.isel(time=-1).plot(ax=axs, cmap='YlGnBu', robust=True)
axs.set_title('test')
plt.tight_layout()
fig.savefig(f'pcrglobwb_RolfTestRhine_discharge_map', bbox_inches='tight', dpi=300)

In [ ]:
pad=.2

# Write data for model output at GRDC location to pandas series,
# Use padding and max to correct for GRDC station location error

#TODO get model output pixel closest to some lat-lon needs to be a util.

dataExpAtGRDCLocation = dataExp.sel(longitude=slice(grdc_lon-pad, grdc_lon+pad), 
                         latitude=slice(grdc_lat-pad, grdc_lat+pad)
                        ).max(['longitude', 'latitude']
                        ).rename('ERA5Exp'
                        ).to_dataframe()

dataExpAtGRDCLocation.index=dataExpAtGRDCLocation.index.to_datetimeindex()

dataRefAtGRDCLocation = dataRef.sel(longitude=slice(grdc_lon-pad, grdc_lon+pad), 
                         latitude=slice(grdc_lat-pad, grdc_lat+pad)
                        ).max(['longitude', 'latitude']
                        ).rename('ERA5Ref'
                        ).to_dataframe()

dataRefAtGRDCLocation.index=dataRefAtGRDCLocation.index.to_datetimeindex()



In [ ]:
# Combine timeseries of ERA-Interim, ERA5 and GRDC observations in a pandas dataframe
df = pd.concat([dataExpAtGRDCLocation, dataRefAtGRDCLocation, grdc_obs],axis = 1)

# Note: the data come at different time stamps
print(df.head())

# For now, interpolating to get similar sets:
df = df.fillna(method='ffill').dropna()
print(df.head())

In [ ]:
fig, ax = plt.subplots(dpi=120)
df.plot(ax=ax)
ax.set_ylim(1000, 10000)
ax.set_ylabel('Streamflow (m$^3$ s$^{-1}$)')
ax.legend(['PCRGlobWB Moselle replaced by MARRMoT-m01','PCRGlobWB normal','Observations from GRDC'])

dataset='ERA5Ref'
yloc=0.6
ax.text(1.02, yloc, f"{dataset}:\n"
        f"NSE: {hm.nse(df[dataset], df['GRDC']):.2f} \n"
        f"KGE (2009): {hm.kge_2009(df[dataset], df['GRDC']):.2f}\n"
        f"SA: {hm.sa(df[dataset], df['GRDC']):.2f}\n"
        f"ME: {hm.me(df[dataset], df['GRDC']):.2f}",
        transform=ax.transAxes, va='top')

fig.savefig(f'pcrglobwb_{basin_name}_hydrograph', bbox_inches='tight', dpi=300)
